In [1]:
import pyspark
import random
import time
import itertools
from collections import OrderedDict
import sys
sc= pyspark.SparkContext()
input_path = "../data/yelp_train.csv"

def min_hash(iterator):
    iterator=list(iterator)
    uid = iterator[1]
    bid = iterator[0]
    hash_val = []
    for i in range(number_of_hash):
        hash_values = []
        for  j in uid:
            hash_values.append((a[i] * j + b[i]) % len(users))
        hash_val.append(min(hash_values))
    return (bid, hash_val)

def check_similarity(p):

    u1=set(u_mat[p[0]])
    u2=set(u_mat[p[1]])
    int= len(u1.intersection(u2))
    union=len(u1.union(u2))
    sim=int/union
    return p,float(sim)

In [2]:
candidate_pairs=set()

rdd=sc.textFile(input_path)
header = rdd.first()
rdd_1=rdd.filter(lambda x:x!= header).map(lambda x:x.split(','))

#no of users and businesss
users= rdd_1.map(lambda x:x[0]).distinct().collect()
business_id=rdd_1.map(lambda x:x[1]).distinct().collect()

dict_busi =OrderedDict()
dict_users=OrderedDict()
for i, v in enumerate(business_id, 0):
    dict_busi[v] = i

for i, v in enumerate(users, 0):
    dict_users[v] = i

pair_busi=list(dict_busi.values())

# columns: [bidx, [uidx1, uidx2, ....]]
rdd_2=rdd_1.map(lambda x:(dict_busi[x[1]],dict_users[x[0]])).groupByKey() 
u_mat=rdd_2.collectAsMap()


a=[]
b=[]
# hash functions
for x in range(150):
    rand=random.randint(0,len(business_id))
    while rand in a:
        rand=random.randint(0,len(business_id))
    a.append(rand)
    while rand in a:
        rand=random.randint(0,len(business_id))
    b.append(rand)
number_of_hash=len(a)

In [3]:
# calculating signature matrix
matrix = rdd_2.map(min_hash)

In [4]:
rdd_2.take(3)

[(12262, <pyspark.resultiterable.ResultIterable at 0x11141e8d0>),
 (12264, <pyspark.resultiterable.ResultIterable at 0x11141e2e8>),
 (0, <pyspark.resultiterable.ResultIterable at 0x11141e198>)]